<a href="https://colab.research.google.com/github/JafarLone/RealTime-Emotion-Sentiment-Detector/blob/main/Copy_of_S_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This will remove all files and folders in the /content directory
!rm -rf /content/*


In [2]:
# Create the project structure with the 'assets' directory for the background image
!mkdir -p my_project/assets


In [3]:
# Upload the background image
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload the image from your local machine


Saving background.webp to background.webp


In [4]:
# Move the uploaded image to the assets folder (adjust the name if it's different)
!mv background.webp my_project/assets/background.png


In [5]:
# Verify the image exists in the assets folder
!ls my_project/assets/


background.png


In [6]:
!pip install streamlit opencv-python-headless deepface mediapipe google-cloud-speech pydub ffmpeg-python SpeechRecognition transformers moviepy && apt-get install -y ffmpeg && npm install -g localtunnel


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.

changed 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


Overwriting /content/my_project/app.py


In [1]:
%%writefile /content/my_project/app.py

import cv2
from deepface import DeepFace
import streamlit as st
import tempfile
import time
from pydub import AudioSegment
import speech_recognition as sr
from transformers import pipeline
import moviepy.editor as mp
import base64
import math

# Initialize the sentiment analysis model
sentiment_analysis = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Function to set background image from local file
def add_bg_from_local(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    st.markdown(
        f"""
        <style>
        .stApp {{
            background-image: url("data:image/png;base64,{encoded_string}");
            background-size: cover;
            background-repeat: no-repeat;
            background-attachment: fixed;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

# Set background using local image
add_bg_from_local('/content/my_project/assets/background.png')

# Custom CSS styling for sidebar and main content area
st.markdown("""
    <style>
    .sidebar .sidebar-content {{
        background-color: #1e1e2d;
        padding: 20px;
    }}
    .main-title {{
        font-size: 2.8em;
        font-weight: bold;
        color: white;
        text-align: center;
        padding-top: 20px;
        padding-bottom: 20px;
        font-family: Arial, sans-serif;
    }}
    </style>
""", unsafe_allow_html=True)

# Title section in the main area
st.markdown("<h1 class='main-title'>Emotion & Sentiment AI</h1>", unsafe_allow_html=True)

# Sidebar for video upload options
st.sidebar.header("Upload or Capture Video")
uploaded_file = st.sidebar.file_uploader(
    "Upload a video", type=["mp4", "mov", "avi"], help="Drag and drop or browse to upload your video."
)

if uploaded_file is not None:
    # Save the uploaded video to a temporary file
    tfile = tempfile.NamedTemporaryFile(delete=False)
    tfile.write(uploaded_file.read())

    # Display the original video (unprocessed)
    st.video(tfile.name)
    st.write("Original video displayed above.")

    # "Analyze Emotion & Sentiment" button
    if st.button("Analyze Emotion & Sentiment"):
        sentiments = []
        sentiment_results = []  # To store detailed sentiment results

        # Extract audio from video
        audio_path = tempfile.mktemp(suffix='.wav')
        video = AudioSegment.from_file(tfile.name)
        video.export(audio_path, format="wav")

        recognizer = sr.Recognizer()
        audio_file = sr.AudioFile(audio_path)
        transcription = ""
        try:
            with audio_file as source:
                recognizer.adjust_for_ambient_noise(source)
                audio_data = recognizer.record(source)
                transcription = recognizer.recognize_google(audio_data)

                # Split transcription into chunks based on a 6-second duration
                total_duration = video.duration_seconds  # Total video duration in seconds
                num_chunks = math.ceil(total_duration / 6)  # Number of 6-second chunks

                # Divide the text transcription into equal chunks
                words = transcription.split()
                words_per_chunk = math.ceil(len(words) / num_chunks)

                for i in range(0, len(words), words_per_chunk):
                    chunk_text = " ".join(words[i:i+words_per_chunk])
                    sentiment = sentiment_analysis(chunk_text[:512])[0]
                    sentiments.append(chunk_text)  # Captions for overlay
                    sentiment_results.append((chunk_text, sentiment))  # Detailed result

        except Exception as e:
            st.write(f"Error in audio processing: {e}")

        # Video processing with OpenCV
        cap = cv2.VideoCapture(tfile.name)
        frame_width, frame_height, fps = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), cap.get(cv2.CAP_PROP_FPS)
        fps = fps or 20
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        processed_frames = []
        progress_bar = st.progress(0)
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Determine the current chunk index based on frame count and FPS
            current_time = frame_count / fps
            chunk_index = int(current_time // 6)  # Each chunk lasts 6 seconds

            if chunk_index < len(sentiments):
                caption_text = sentiments[chunk_index]

                # Overlay transcription text (live captions) on the frame
                cv2.putText(frame, f"{caption_text}", (10, frame_height - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

            # Optional: Add emotion detection
            try:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                result = DeepFace.analyze(rgb_frame, actions=["emotion"], enforce_detection=False)
                emotion = result[0]["dominant_emotion"] if isinstance(result, list) else result["dominant_emotion"]

                # Draw emotion and face bounding box
                if "region" in result[0]:
                    face_area = result[0]["region"]
                    x, y, w, h = face_area["x"], face_area["y"], face_area["w"], face_area["h"]
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, f"Emotion: {emotion}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            except Exception as e:
                cv2.putText(frame, "Emotion detection failed", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            processed_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            frame_count += 1
            progress_bar.progress(frame_count / total_frames)

        cap.release()
        progress_bar.empty()

        # Save processed video
        output_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4").name
        video_clip = mp.ImageSequenceClip(processed_frames, fps=fps)
        video_clip.write_videofile(output_path, codec="libx264")

        # Display processed video
        st.write("Processed Video with Live Captions and Emotions:")
        st.video(output_path)

        # Display detailed sentiment analysis results
        st.write("Sentiment Analysis Results:")
        for idx, (chunk_text, sentiment) in enumerate(sentiment_results):
            st.write(f"Sentiment {idx + 1}: {chunk_text}")
            st.write(f"Sentiment Result: {sentiment['label']}")
            st.write("---")


Overwriting /content/my_project/app.py


In [2]:
!wget -q -O - ipv4.icanhazip.com


34.80.37.239


In [ ]:
# Run the app and open it using Localtunnel
!streamlit run my_project/app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.37.239:8501

your url is: https://weak-breads-cough.loca.lt
2024-11-17 10:59:43.281038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-17 10:59:43.312303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-17 10:59:43.321519: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 10:59:43.342298: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr